# A Deep dive into the Atomic Agents framework.
This series aims to provide an in-depth understanding of each building block within the Atomic Agents framework, starting with the BaseAgent class. 
Whether you're a seasoned developer or new to AI agent development, this series will offer valuable insights into how to effectively utilize and extend the Atomic Agents framework for your AI applications.

# What is the Atomic Agents Framework?
The Atomic Agents framework is a new approach to AI development, inspired by the principles of Atomic Design. This framework breaks down AI systems into smaller, self-contained, and reusable components, much like LEGO blocks. This modularity ensures that AI development is both flexible and predictable, allowing developers to create complex AI applications with ease and confidence.

# Why a BaseAgent?
The BaseAgent class serves as the foundational building block for creating AI agents within the Atomic Agents framework. It encapsulates essential functionalities such as handling user inputs, generating responses, and managing chat history. By dissecting this class, we aim to uncover the "magic" behind its operations and provide a clear roadmap for setting up and customizing your own AI agents

If you just want to get started checkout the quickstart notebook: https://github.com/KennyVaneetvelde/atomic_agents/blob/main/examples/notebooks/quickstart.ipynb

# BaseAgent
Let's have a look at what the `BaseAgent` provides.
This class provides the core functionality for handling chat interactions, including managing memory, generating system prompts, and obtaining responses from a language model.

#### Attributes
- **input_schema** (Type[BaseAgentIO]): Schema for the structure of the input data.
- **output_schema** (Type[BaseAgentIO]): Schema for the structure of the output data.
- **client**: Client for interacting with the language model.
- **model** (str): The model to use for generating responses.
- **memory** (**AgentMemory**): Memory component for storing chat history of the agent.
- **system_prompt_generator** (**SystemPromptGenerator**): Component for generating system prompts.
- **initial_memory** (**AgentMemory**): Initial state of the memory.

#### Public Functions
- **reset_memory()**: Resets the memory to its initial state.
- **get_response()**:  Obtains a response from the language model.
- **run()**:  Runs the chat agent with the given user input.
- **get_context_provider()**:  Retrieves a context provider by name.
- **register_context_provider()**: Registers a new context provider.
- **unregister_context_provider()**:  Unregister an existing context provider.

#### Private Functions:
- **get_and_handle_response()**: Handles obtaining and processing the response.
- **init_run()**: Initializes the run with the given user input. 
- **pre_run()**: Prepares for the run. This method can be overridden by subclasses to add custom pre-run logic.
- **post_run()**: Finalizes the run with the given response.

Here is an overview of what we have to define to be able to create an agent.
To create the `BaseAgent` we need to prepare all the parts we need to configure and instantiate the Agent.

![BaseAgent overview schema](../assets/baseAgent.png 'BaseAgent overview schema')

So let's get starting, first we need to install the packages we need.

In [ ]:
# Install the necessary packages
%pip install atomic-agents openai instructor Groq

Next we will do the nessesary imports

In [ ]:
# we need to be able to load an API_key from a .env file 
from dotenv import load_dotenv
# we need the os to get to out environment variables
import os
# we need to be able to provide a date
import datetime
# we need instructor to interact with the model
import instructor
# we need groq because it's free and easy to setup
from groq import Groq

# we need the AgentMemory because this is the memory of the agent
from atomic_agents.lib.components.agent_memory import AgentMemory
# we need the BaseAgent because it's the core of the framework 
# we need the BaseAgentConfig to create the configuration for the BaseAgent
from atomic_agents.agents.base_agent import BaseAgent, BaseAgentConfig
# we need the SystemPromptInfo to create our system prompt
# we need the SystemPromptGenerator to generate the system prompt based on the SystemPromptInfo
# we need the SystemPromptContextProviderBase to create a context provider by extending that class
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator, SystemPromptInfo, SystemPromptContextProviderBase

# we need the console to be able to print and have a peek into the magic
from rich.console import Console
# we need Markdown to convert the system prompt into a nice format to read in the console
from rich.markdown import Markdown
# we need the console for obvious reasons
console = Console()
# we need to load the environment variables from the .env file
load_dotenv()

So lets start with defining a system prompt, to create a system prompt you will need the `SystemPromptInfo`. 

# SystemPromptInfo
In the `SystemPromptInfo` we will define everything we need to create a system prompt.

## Arguments
- **background**: this is the initial description of the agent and what its purpose is. 
This is the place to define the role the agent should behave on. 
Keep in mind its not really a rol its more telling the Agent to narrow the topics it should use to search for in its corpus.

- **steps**: these are the internal assistant steps you want the agent to take.

- **output_instructions**: this is where you define how the agent should answer the user.

- **context_providers**: this can be any provider you want to add to the system prompt. 
It can be a date, some lorumIpsum, the model you are using, whatever extra context you want to provide to the system prompt and the agent needs to know of.

In [ ]:
# Define the SystemPromptInfo object
system_prompt_info = SystemPromptInfo(    
    background=[
        'This assistant is a general-purpose AI designed to be helpful and friendly.',
    ],
    steps=[
        'Understand the user input.',
        'Reason about the input.',
        'Respond to the user.',
    ],
    output_instructions=[
        'Provide helpful and relevant information to assist the user.',
        'Be friendly and respectful in all conversations.',
        'Always use the available additional information and context to enhance the response',
    ],
)

So lets have a look and see what's defined in the `SystemPromptInfo`.

In [27]:
# log the SystemPromptInfo 
console.print(system_prompt_info)

SystemPromptInfo(
    background=['This assistant is a general-purpose AI designed to be helpful and friendly.'],
    steps=['Understand the user input.', 'Reason about the input.', 'Respond to the user.'],
    output_instructions=[
        'Provide helpful and relevant information to assist the user.',
        'Be friendly and respectful in all conversations.',
        'Always use the available additional information and context to enhance the response'
    ],
    context_providers={}
)

# Context Providers
As mentiond a context provider can be anything you want to be added to the system prompt. 
So lets add a data so the agent knows what day it is when we have a conversation.
To create a context provider we need to use the `SystemPromptContextProviderBase` class and extend it.
We will create a context provider to provide a Date to each system prompt and provide a format attribute so you can change the date format the way you want.

## Attributes
- **Title**: the title for your context provider, this can be whatever you want it to be named.

## Functions
- **get_info()**: is a function you can overwrite to return whatever you define you want to get from your context provider

In [ ]:
# Let's extend the SystemPromptContextProviderBase
class CurrentDateContextProvider(SystemPromptContextProviderBase):
    def __init__(self, format: str = '%Y-%m-%d %H:%M:%S', **kwargs):
        super().__init__(**kwargs)
        self.format = format

    def get_info(self) -> str:
        return f'Date: {datetime.datetime.now().strftime(self.format)}' # **Remember** 'title' is from the base class we are extending.

In [ ]:
# Create an instance of the new CurrentDateContextProvider, 
# # keep in mind that now we do have to pass the title attribute from the base class because its required.
provider = CurrentDateContextProvider(title='Datetime')

# Call the get_info() method on the instance and print the result.
console.print(provider.get_info())

Now that we have defined a new `CurrentDateContextProvider` we can add it to the `SystemPromptInfo` context_providers attribute. 

Do know that you could also register and unregister a context provider straight on the BaseChatAgent. We will see this later, but for now we pass it to the `SystemPromptInfo`.

In [ ]:
# Add the new CurrentDateContextProvider to the SystemPromptInfo
system_prompt_info.context_providers = {
    'date': CurrentDateContextProvider(
        title='Datetime Context Provider', 
        format='%Y-%m-%d %H:%M:%S'
    )
}
console.print(system_prompt_info)

# SystemPromptGenerator
Now we are ready to generate a system prompt, to do so we need the `SystemPromptInfo` and the `SystemPromptGenerator`.

Remember we imported the `Markdown`, so we are using it to give our console logs a nice format, it's not needed to work with agents because the framework does it by default for the system prompt. 
We do it for the readability in our console.

## Arguments
- **system_prompt_info**: which holds background, steps, output_instructions and context_providers

## Functions
- **generate_prompt()**: will generate a system prompt from the `SystemPromptInfo` to be used by the agent

In [ ]:
# define the SystemPromptGenerator with the SystemPromptInfo object
system_prompt_generator = SystemPromptGenerator(system_prompt_info)
# log the system prompt
console.print(Markdown(system_prompt_generator.generate_prompt()))

# Recap
1. We created a `SystemPromptInfo` config object, 
2. We created the `CurrentDateContextProvider` context provider.
3. We added the `CurrentDateContextProvider` to the `SystemPromptInfo`.
4. We defined a `SystemPromptGenerator` that used the `SystemPromptInfo`.

The next step is adding some memory to the agent to keep track of it's conversation.

# AgentMemory
The `AgentMemory` is used by the agent to keep track of the conversation. Later when you want to use multiple agent you could even share this memory with other agents for example. 
To add to a `AgentMemory` we make use of a `Message` with a predefined format.

## Attributes:
- **History**:  A list of messages representing the chat history.

## Functions
- **add_message()**: A message has several arguments:
    - **role** (str): The role of the message sender, user or agent.
    - **content** (str): The content of the message.
    - **tool_message** (Optional[Dict]): Optional tool message to be included when making use of a tool
    - **tool_id** (Optional[str]): Optional unique identifier for the tool call when making use of a tool

- **get_history()**:Retrieves the chat history.

- **dump()**: Converts the chat history to a list of dictionaries.

- **load()**:  Loads the chat history from a list of dictionaries.

- **copy()**:  Creates a copy of the chat memory.

In [ ]:

# Define initial memory with a greeting message from the assistant
initial_memory_message = [
    {'role': 'assistant', 
     'content': 'How do you do and what can I do for you today?',
     'tool_message': 'no tool used',
     'tool_id': None
    } 
]
# Initialize the agent memory to store conversation history
agent_memory = AgentMemory()

# Load the initial memory into the agent memory
agent_memory.load(initial_memory_message)

# examples
console.print(agent_memory.dump())
console.print(agent_memory.get_history())

The same as before we need to start with a config object which in this case is the `BaseAgentConfig` so let's start defining everything we need for the `BaseAgentConfig`.

First we need a Client but to configure this client we first need to add our API_key and as mentioned at the start we will use Groq. 
If you don't have an account yet now is the time to create one and get your API_Key. 
You could also use OpenAI or Anthropic whatever you want, you can use the Instructor library for this. 
You could even use a local Ollama server and use that if you have a beefy PC to run your models on. 

Let's start with the API_Key.
For safety and good organization use a .env file to store your keys and make sure to add it to your .gitignore file before commiting your notebook.

In [ ]:
# get your API_Key from your enviroment variable if it's not there add your API_Key here 
# Keep in mind to not make this code public with your key
API_KEY = ''
if not API_KEY:
    # get the environments variable
    API_KEY = os.getenv('GROQ_API_KEY') # OPENAI_API_KEY, OLLAMA_API_KEY, ...
    
if not API_KEY:
    raise ValueError('API key is not set. Please set the API key as a static variable or in an environment variable.')

Now we will define the client using the `instructor` library for Groq.

For more details visit Instructor at: https://github.com/jxnl/instructor 

In [ ]:
client = instructor.from_groq(
    Groq(api_key=API_KEY,),
    mode=instructor.Mode.TOOLS
)

# Input_schema / Output_schema
Lets discuss the input/output schema's and why they are different then the input/output_instructions. 
First of all the instructions are part of the `SystemPrompt` and will be provide via the `SystemPromptInfo`.

- `output_instruction` = this is the place where you want to define the agent and how it should behave. You can mention what type of agent it is or which role you want it to act on. this is where you want to tell how the agent to response, be polite, be helpful, guide me step by step or ask a new question after every answer you provide.


The schema's are part of the agent itself and will be provided via the `BaseAgentConfig`.
The schema's are about the data structure, for example if you want to receive an answer in a formated structure or provide input based on the answer of 2 other agents this is the place.
- `input_schema` = you need the response of 2 other agents to feed this agent then you would define this in the input_schema
- `output_schema` = you want a formated output like a blog article which will always nee to have a Title, Summary, Numbers, Story, Sources, Keywords, Hashtags then you would define this in the output_schema. You could also define things like you want at least 3000 words as a response or exactly 5 results.

So now that we know the difference we can determin that in this case we don't need the the schema's but only the instruction.

# Create an AI Agent

So we came to the final part of this notebook, the part you were waiting for to create an Agent after a deep dive in the `BaseAgent` Class and everything that is needed.
Now that we have created all the seperate blocks, we can now configure the Agent and We will create the main loop of the application. 

We will also add some code to be able to exit the conversation when typing `/exit` or `/quit`. 
Use this if you want to go to the last part of the notebook.

In [ ]:

# configure the agent
agent = BaseAgent(
    config=BaseAgentConfig(
        client=client,
        system_prompt_generator=system_prompt_generator,
        model='llama3-70b-8192', # Specify the model you want to use
        memory=agent_memory
    )
)

# Main loop for testing the chat agent
# take the 1st message from the agentMemory and prefix with the role: Agent
console.print(f'Agent: {initial_memory_message[0]["content"]}')

while True:
    # get the user inpput and prefix with the role: User
    user_input = input('User: ')
    print(f'User: {user_input}')
    
    if user_input.lower() in ['/exit', '/quit']:
        print('Exiting chat, see you later ...')
        break
    
    # run the agent and asign the user input to the conversation
    response = agent.run(agent.input_schema(chat_message=user_input))
    print(f'Agent: {response.chat_message}')

Now you can start a conversation with Llama3 via Groq.
To test the context provider ask the agent for the date: 'What day is it today?'
You will notice that the agent is aware of the date. 

`Agent: Today's date is 2024-07-07`

Without the context provider it will tell you it's not aware of time.

`Agent: I'm happy to help! However, I'm a large language model, I don't have have access to real-time information, including the current date.`

In [ ]:
# lets play 
# get the context provider
console.print('# get the context provider #')
console.print(agent.get_context_provider(provider_name="date"))

# get the system prompt
console.print('# get the system prompt with context provider#')
console.print(f'system prompt = {agent.system_prompt_generator.generate_prompt()}')

# unregister the context provider
console.print('# Unregister the context provider #')
agent.unregister_context_provider('date')

# get the system prompt
console.print('# get the system prompt now without context provider #')
console.print(f'system prompt = {agent.system_prompt_generator.generate_prompt()}')

# register the context provider
console.print('# register the context provider #')
agent.register_context_provider('date', CurrentDateContextProvider(
        title='Datetime Context Provider', 
        format='%Y-%m-%d %H:%M:%S')
)

# get the system prompt
console.print('# get the system prompt again with context provider #')
console.print(agent.system_prompt_generator.generate_prompt())


# Conclusion
Congratulations on completing this deep dive into the BaseChatAgent class! 
By now, you should have a solid understanding of its internal workings, setup requirements, and how to effectively utilize and extend it for your AI applications. This foundational knowledge will serve as a crucial stepping stone as we continue to explore the Atomic Agents framework.

# What's Next?
In the next notebook of this series, we will shift our focus to another essential component of the Atomic Agents framework: Tools. We will explore how to create and use these tools to enhance the capabilities of your AI agents. Specifically, we will cover:

- The purpose and functionality of Tools within the framework.
- Step-by-step instructions on how to create custom Tools.
- Practical examples demonstrating how to integrate and utilize Tools in your AI projects.

Stay tuned for an exciting journey into the world of Tools, where we will unlock even more potential for your AI agents. Thank you for following along, and we look forward to seeing you in the next notebook!


For a complete script check it out at:  
/scripts/A_deep_dive_into_atomic_agents-BaseAgent.py
